In [ ]:
import numpy as np
from scipy.special import erf
from scipy.stats import ttest_rel
import time

# ───────────────────────────────────
# Standard GELU function
# ───────────────────────────────────
def gelu_np(x: np.ndarray) -> np.ndarray:
    return 0.5 * x * (1.0 + erf(x / np.sqrt(2.0)))

# ───────────────────────────────────
# Cached GELU with precomputed table
# ───────────────────────────────────
class CachedGELU:
    def __init__(self, x_min=-100.0, x_max=100.0, N=20000):
        self.x_min = float(x_min)
        self.x_max = float(x_max)
        self.N = int(N)
        self.step = (self.x_max - self.x_min) / (self.N - 1)
        self.inv_step = 1.0 / self.step
        x_table = np.linspace(self.x_min, self.x_max, self.N, dtype=np.float32)
        self.y_table = 0.5 * x_table * (1.0 + erf(x_table / np.sqrt(2.0))).astype(np.float32)
        self.slope = np.empty_like(self.y_table)
        self.slope[:-1] = np.diff(self.y_table)
        self.slope[-1] = self.slope[-2]

    def apply(self, X: np.ndarray) -> np.ndarray:
        Xf = np.ascontiguousarray(X, dtype=np.float32)
        idx_f = (Xf - self.x_min) * self.inv_step
        idx = idx_f.astype(np.int32)
        np.clip(idx, 0, self.N - 1, out=idx)
        idx_f -= idx
        out = self.y_table[idx] + self.slope[idx] * idx_f
        mask_outside = (Xf < self.x_min) | (Xf > self.x_max)
        if np.any(mask_outside):
            X_out = Xf[mask_outside]
            out[mask_outside] = 0.5 * X_out * (1.0 + erf(X_out / np.sqrt(2.0)))
        return out

# ───────────────────────────────────
# Benchmark utility (returns full timing list)
# ───────────────────────────────────
def benchmark_times(fn, data, runs=50):
    fn(data)  # warm-up
    times = []
    for _ in range(runs):
        t0 = time.time()
        fn(data)
        times.append((time.time() - t0) * 1e3)  # milliseconds
    return np.array(times)

# ───────────────────────────────────
# Main script
# ───────────────────────────────────
if __name__ == "__main__":
    np.random.seed(42)
    size = 100_000_000
    X = np.random.uniform(-100, 100, size).astype(np.float32)

    # Create GELU and CachedGELU functions
    fast = CachedGELU()

    # Benchmark both (50 runs)
    ref_times = benchmark_times(gelu_np, X)
    fast_times = benchmark_times(fast.apply, X)

    ref_mean, ref_std = np.mean(ref_times), np.std(ref_times)
    fast_mean, fast_std = np.mean(fast_times), np.std(fast_times)

    # Compute reference output ONCE for accuracy check
    ref_output = gelu_np(X)
    cached_output = fast.apply(X)

    # Error analysis
    max_err = np.max(np.abs(ref_output - cached_output))
    mean_err = np.mean(np.abs(ref_output - cached_output))

    # Paired t-test for p-value
    t_stat, p_value = ttest_rel(ref_times, fast_times)

    # Print results
    print("\n--- GELU Timing Results (100M values, 50 runs) ---")
    print(f"Standard GELU (erf):  {ref_mean:8.2f} ms ± {ref_std:6.2f}")
    print(f"CachedGELU         :  {fast_mean:8.2f} ms ± {fast_std:6.2f}")
    print(f"Speed-up factor    :  {ref_mean / fast_mean:8.2f} ×")
    print(f"Max abs error      :  {max_err:.6e}")
    print(f"Mean abs error     :  {mean_err:.6e}")
    print(f"p-value (paired t-test): {p_value:.4e}")
